# Course 2. Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization

## Practical aspects of Deep Learning

### Train / Dev / Test sets

List of hyperparameters:
* Number of layers
* Number of hidden units in each layer
* Learning rates
* Regularization parameters
* Activation functions

In order to figure out what are the most suitable hyperparameter values for your neural net it is advised that you constantly experiment with different values and iterate on repeating the experiments.

Previous era of machine learning was characterized by the fact that we had much smaller amounts of data. 
* 70 / 30 train / test split or 60 / 20 / 20 train / dev / test split was ok when you have up to 10000 examples in your dataset, but when you have 1m examples 98 / 1 / 1 is also good choise
* Make sure that dev and test set come from the same distribution
* Not having the test set might be ok

### Bias and Variance

<img src="imgs/biasvariance.png">

It is possible that classifer has both high bias and high variance and that would look something like this:

<img src="imgs/highbias&highvariance.png">


### Basic recipe for machine learning

In [1]:
high_bias = True # measure error on train set 
high_variance = True # measure error on dev / test set

if high_bias:
    print("* Try bigger network")
    print("* Train longer")
    print("* Search different nn architecture")

if high_variance:
    print("* Find more data or create more features")
    print("* Regularization")
    print("* Search different nn architecture")


* Try bigger network
* Train longer
* Search different nn architecture
* Find more data or create more features
* Regularization
* Search different nn architecture


### Regularization

Let introduce regularization on an example of Logistic Regression.

L2 regularization:
\begin{align}
J(W, b) = \frac{1}{m}\sum_{i = 1}^{m}L(y^{(i)}, \hat{y}^{(i)}) + \frac{1}{2m}\sum_{j = 1}^{n_x} {\lVert w_j \rVert}^2
\end{align}

L1 regularization:
\begin{align}
J(W, b) = \frac{1}{m}\sum_{i = 1}^{m}L(y^{(i)}, \hat{y}^{(i)}) + \frac{1}{m}\sum_{j = 1}^{n_x} {\lVert w_j \rVert}
\end{align}

In general, L2 regularization is used much much more often. L1 regularization is used when we want to make $W$ sparse.

In the case of Neural Networks L2 regularization would look like this:
\begin{align}
J(W_1, b_1, W_2, b_2 ... W_L, b_L) = \frac{1}{m}\sum_{i = 1}^{m}L(y^{(i)}, \hat{y}^{(i)}) + \frac{1}{2m}\sum_{l = 1}^{L} {\lVert W_l \rVert}^2 = \frac{1}{m}\sum_{i = 1}^{m}L(y^{(i)}, \hat{y}^{(i)}) + \frac{1}{2m}\sum_{i = 1}^{n_{l - 1}}\sum_{j = 1}^{n_l} {\lVert w_{ij} \rVert}^2
\end{align}

because $W$'s are always $n_{l-1}$ x $n_l$ dimension.

Note that L2 norm is also called Frobenius norm.

### Why Regularization Reduces Overfitting

Couple of good points .. watch it again
https://www.youtube.com/watch?v=NyG-7nRpsW8&index=5&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc

### Dropout regularization

Dropout is a efficient way of regularizing neural network paramenters.

Let say this is our neural network and what we are going to do is that for each layer we are going to set the probability of keeping (eliminating) each node in that layer. Then, when we do training, after one training example we decide to remove certain nodes (by removing all in-going and out-going links of that node) so we end up with much smaller network. After we do backpropagation new training instance comes along and then we choose different nodes to be eliminated and so on.

<img src="imgs/dropout1.png">
<img src="imgs/dropout2.png">

The parameter that represents probability of keeping each node is called `keep-prob`. There are several ways to implement dropout regularization but the most common one is `Inverted dropout`.

In [2]:
# inverted dropout for layer 3, keepProb = 0.8
import numpy as np
keepProb = 0.8
a3 = np.random.rand(4, 1)
d3 = np.random.rand(a3.shape[0], a3.shape[1]) < keepProb
a3 = np.multiply(a3, d3) # element-wise multiplication
a3 /= 0.8
# z4 = w4 * a3 + b4
print(d3)
print(a3)


[[ True]
 [ True]
 [False]
 [ True]]
[[ 0.17275606]
 [ 1.06163271]
 [ 0.        ]
 [ 0.49236518]]


We don't use dropout when making a prediction. More info and the math behind dropout can be found it Hintons paper http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf.

### Understanding dropout

Dropout works really well in computer vision problems and it is used almost as a default because you almost never have enough data and model is usually overfitting. In other areas of ai is not used that frequently. One downside of dropout is that cost function J may not be monotonically decreasing. Because of that prefered way of debbuging the code is to plot cost function J without dropout (keep-prob is set to 1) and if everything is ok, switch on dropout by setting keep-prob to `0.8` or some other value.


### Other regularization methods

Other regularization methods include:
1. Data augmentation - flipping horizontally, random cropping, adding distorion transformations etc.
2. Early stopping - During the training we usually plot train loss and val loss and what we can sometimes encounter is that at some point val loss starts to grow comparing to train loss. At this point we may want to stop the training because that will probably give us best results. Early stopping is a technique for stopping the model training at this point instead of continuing training.

<img src="imgs/earlystopping.png">

3. Orthogonalization - is a concept that its good to keep in mind. The idea is that we treat training of the model as a 2 separate tasks. One is finding the right set of parametars to make sure that model is learning and second is to make sure not to overfit by introducing regularizations etc.


### Normalizing Inputs

Let $X = [x_1, x_2, ..., x_n] \in R^{m \times n}$ be our data, where $x_1, ...x_n$ are feature vectors. You can imagine that our data looks like this for $n = 2$:

<img src="imgs/normalization1.png">

We first subtract the mean of each feature from $X - \mu(X) = [x_1 - \frac{1}{m}\sum_{i = 1}^{m}x_{i, 1}, .. , x_n - \frac{1}{m}\sum_{i = 1}^{m}x_{i, n}]$ and by doing that we are centering our data:

<img src="imgs/normalization2.png">

And then, devide by standard deviation $\frac{X - \mu(X)}{\sigma^2(X)} = [\frac{x_1 - \frac{1}{m}\sum_{i = 1}^{m}x_{i, 1}}{\frac{1}{m}\sum_{i = 1}^{m}x^2_{i, 1}}, .. , \frac{x_n - \frac{1}{m}\sum_{i = 1}^{m}x_{i, n}}{\frac{1}{m}\sum_{i = 1}^{m}x^2_{i, n}}]$ where now variance of all features is equal to 1:

<img src="imgs/normalization3.png">

__Variance__ measures how far a set of (random) numbers are spread out from their average value (mean). And __standard deviation__ is squared variance.

### Vanishing and exploading gradients

When training very deep neural network we can encounter problem where gradients are too small (vanishing) or too big (exploading) which makes the training difficult. Problem of vanishing / exploading gradients can be partialy solved by carefull weight initialization depending on what activation function are you using.

### Weight Initialization in a Deep Network

1) If you are using $ReLU$ weights should be initialized to (he-et-al initialization https://arxiv.org/abs/1502.01852) and what it does is basically setting the variance of $w$ to $\frac{2}{n_l}$


In [3]:
import numpy as np
layer_size = [5, 4]
l = 1
w=np.random.randn(layer_size[l],layer_size[l-1])*np.sqrt(2/layer_size[l-1])

2) If you are using $Tanh$ weights should be initialized to (Xavier initialization http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)


In [4]:
import numpy as np
layer_size = [5, 4]
l = 1
w=np.random.randn(layer_size[l],layer_size[l-1])*np.sqrt(1/layer_size[l-1])

or this (Bengio initialization):

In [5]:
import numpy as np
layer_size = [5, 4]
l = 1
w=np.random.randn(layer_size[l],layer_size[l-1])*np.sqrt(2/(layer_size[l] + layer_size[l-1]))

### Numerical aproximation of gradients

Derviatives can be aproximated with $f'(\theta) \approx \frac{f(\theta + \varepsilon) - f(\theta - \varepsilon)}{2\varepsilon}$

<img src="imgs/numerical_grad_aprox.png"> 

### Gradient checking

Gradient checking can be of great help when implementing backpropagation. It can be done in a following way:
1. copy all neural network parametars into a single vector and do the same thing for derviatives of each parametar
2. iterate through all parameters and for each compute approximation of the derivative using approximation formula on previous slide
3. compare vector of approximate derivatives with original vector of derivatives 

<img src="imgs/grad_check.png"> 

### Gradient checking implementation notes

<img src="imgs/grad_check_notes.png">

## Optimization algorithms

### Mini-batch gradient descent

__Batch gradient descent__ means that we are processing entire training set in single iteration (`batch_size = m`). 

__Mini-batch gradient descent__ means that we are processing `batch_size` number of samples in a single iteration where `batch_size` is less then `m`.

__Stochastic gradient descent__ means that we are processing single sample in a single iteration (`batch_size = 1`)

### Understanding mini-batch gradient descent

<img src="imgs/mini_batch_gradient_descent.png">

### Exponentially weighted averages

<img src="imgs/exponentially_weighted_avg.png">

### Understanding exponentially weighted averages



## Tuning process

* Don't use grid search for hyperparameter search. Instead, use random sampling over the range of hyperparameters.
* Consider implementing zoom in of the region that gives the best results and then repeat the process with ranges set to values that are within region that gave the best results

## Tensor flow

Nice basic tutorial ... very basic though